# Overview

Intermediate level

Question #1 (Anova version)
Is there any statistical difference between groups? 
(Light, medium and dark lesions)

# Table of Content

1. [Setup](#setup_cell)
2. [Data Loading](#loading)
3. [Analysis](#analyze)
4. [Discussion](#discuss)

# Setup <a id='setup_cell'></a>
Import the appropriate packages

In [2]:
import PIL
import cv2
import os
from os import listdir
from os.path import isfile, join
import json
import numpy as np
import seaborn as sns # pip install -U seaborn
from matplotlib import pyplot as plt
import pandas as pd
import glob
from IPython.display import display
from scipy.stats import f_oneway

# Data Loading <a id='loading'></a>
Load the data similar to the novice level

In [3]:
directory = os.getcwd().split("\\")[:-2]
new_directory = ""
for component in directory:
    new_directory += component
    new_directory += "\\"

if 'sample_imgs' in listdir(new_directory):
    img_filepaths = glob.glob('../../sample_imgs/*.jp*')
    seg_filepaths = glob.glob('../../sample_segs/*.png')
    dsc_filepaths = glob.glob('../../sample_dscs/*')

In [4]:
# Checks to make sure each image corresponds to it's segmentation
for i in range(len(img_filepaths)):
    
    image_number = int(img_filepaths[0].split("_")[-1].split(".")[0])
    segmentation_number = int(seg_filepaths[0].split("_")[2])
    
    if image_number != segmentation_number:
        print("Error in file order")
        break

# Analysis <a id='analyze'></a>

In [5]:
def get_lesion_pixels(r, g, b):
    
    non_black_r = []
    non_black_g = []
    non_black_b = []

    for row in range(r.shape[0]):

        for col in range(r.shape[1]):

            if r[row][col] != 0:

                if g[row][col] != 0:

                    if b[row][col] != 0:

                        non_black_r.append(r[row][col])
                        non_black_g.append(g[row][col])
                        non_black_b.append(b[row][col])
    return [non_black_r, non_black_g, non_black_b]

In [6]:
all_image_colors = []
for i in range(12):
    
    # Grab orignal image and segmented version 
    color =  PIL.Image.open(img_filepaths[i]).resize((400,300))
    segged = PIL.Image.open(seg_filepaths[i]).resize((400,300))

    # Get blank background
    np_im = np.zeros((300,400))
    backtorgb = cv2.cvtColor(np.float32(np_im),cv2.COLOR_GRAY2RGB)

    random_array = backtorgb * 255
    random_array = random_array.astype(np.uint8)
    sam = PIL.Image.fromarray(random_array)

    # Copy original picture on blank background
    back_im = sam.copy()
    back_im.paste(color, (0, 0), segged)
    im_matrix = np.array(back_im)

    # Get r g b from image
    r= im_matrix[:,:,0]
    g= im_matrix[:,:,1]
    b= im_matrix[:,:,2]

    # Only use the non black pixels
    non_black_pixels = get_lesion_pixels(r,g,b)
    non_black_r = non_black_pixels[0]
    non_black_g = non_black_pixels[1]
    non_black_b = non_black_pixels[2]
    
    # Get rgb means
    r_mean = int(np.average(non_black_r))
    g_mean = int(np.average(non_black_g))
    b_mean = int(np.average(non_black_b))

    # Turn rgb into Hex to find color as a number
    hex_value = '%02x%02x%02x' % (r_mean, g_mean, b_mean)
    
    average_color = int(hex_value,16)
    all_image_colors.append(average_color)

In [7]:
# Order the images from lightest to darkest and split how you want 
all_image_colors = sorted(all_image_colors)
f_oneway(all_image_colors[:4], all_image_colors[4:8], all_image_colors[8:])

F_onewayResult(statistic=10.945525107995786, pvalue=0.0038890565839853203)

# Discussion <a id='discuss'></a>
Did you find any statistical difference? <br>
What were the intervals you used?